# How it started 
So, inspired by this piece by Lazaro Gamio in the [NYT](https://www.nytimes.com/interactive/2020/03/15/business/economy/coronavirus-worker-risk.html), I wanted to figure out whether I could do a similar analysis for Germany. It is possible, but I had to connect the US data for workers physical proximity with the German labour market data, as there is no analysis in Germany of working contexts on this level of granularity ( I asked three experts and googled half a day to confirm this). So, I had to "translate" US codes to German codes. Here is how I did this.

In [2]:
#import a bunch of go-to-libraries
import pandas as pd
import numpy as np
import altair as alt
import scipy.stats as stats
import scipy

Then we need to get the US proximity data from O*\NET by downloading it [from this page](https://www.onetonline.org/find/descriptor/result/4.C.2.a.3/Physical_Proximity.csv?fmt=csv). And then we load it in here.

In [3]:
contactwith_others=pd.read_csv("../RawData/Physical_Proximity.csv")
contactwith_others

,Context,Code,Occupation
0,100,27-2032.00,Choreographers
1,100,29-2021.00,Dental Hygienists
2,100,29-1123.00,Physical Therapists
3,100,29-1069.11,Sports Medicine Physicians
4,99,31-9091.00,Dental Assistants
...,...,...,...
962,17,45-3021.00,Hunters and Trappers
963,14,45-4022.00,Logging Equipment Operators
964,14,27-3043.05,"Poets, Lyricists and Creative Writers"
965,9,27-1013.00,"Fine Artists, Including Painters, Sculptors, a..."


We want to reduce the complexity before we transalte codes. So we use the first 5 digits (including the hyphen) to summarize some jobs, [as the first 5 digits reflect the "broad group" classification](https://www.bls.gov/soc/2018/soc_structure_2018.pdf)

In [4]:
contactwith_others["main_groups_code"]=contactwith_others["Code"].str[:6]
# write the list of broad_groups/main_groups to a list
listofmaingroups=list(contactwith_others["main_groups_code"].unique())
print(len(listofmaingroups))

438


In [5]:

groudict={}
index=0#now iterate through this list of broadgroups

for i in listofmaingroups:
    #for each maingroup we create a list for a row 
    entrieslist=[]
    #we create a list with the jobs listed under this code and the code itself
    joblist=[(list(contactwith_others[contactwith_others["main_groups_code"]==i]["Occupation"])),i]
    #and we create a list with the mean, std etc. metrics for this group of jobs
    metriclist=((contactwith_others[contactwith_others["main_groups_code"]==i]["Context"].describe().tolist()))
    #then we push both these list into our rowlist
    entrieslist.extend(joblist)
    entrieslist.extend(metriclist)
    print(entrieslist)
    #and save it under the indexnumber in the dictionary
    groudict[index]=entrieslist
    #and increase the index by 1
    index+=1
groudict

[['Choreographers', 'Dancers'], '27-203', 2.0, 99.0, 1.4142135623730951, 98.0, 98.5, 99.0, 99.5, 100.0]
[['Dental Hygienists'], '29-202', 1.0, 100.0, nan, 100.0, 100.0, 100.0, 100.0, 100.0]
[['Physical Therapists', 'Radiation Therapists', 'Occupational Therapists', 'Exercise Physiologists', 'Respiratory Therapists', 'Music Therapists', 'Low Vision Therapists, Orientation and Mobility Specialists, and Vision Rehabilitation Therapists', 'Recreational Therapists', 'Speech-Language Pathologists', 'Art Therapists'], '29-112', 10.0, 88.9, 8.5693251387337, 73.0, 84.5, 90.5, 94.5, 100.0]
[['Sports Medicine Physicians', 'Urologists', 'Dermatologists', 'Obstetricians and Gynecologists', 'Surgeons', 'Anesthesiologists', 'Hospitalists', 'Ophthalmologists', 'Family and General Practitioners', 'Physical Medicine and Rehabilitation Physicians', 'Internists, General', 'Pediatricians, General', 'Allergists and Immunologists', 'Neurologists', 'Nuclear Medicine Physicians', 'Psychiatrists', 'Radiologists

[['Bus Drivers, School or Special Client', 'Bus Drivers, Transit and Intercity'], '53-302', 2.0, 81.0, 4.242640687119285, 78.0, 79.5, 81.0, 82.5, 84.0]
[['Gaming Supervisors', 'Slot Supervisors'], '39-101', 2.0, 82.0, 2.8284271247461903, 80.0, 81.0, 82.0, 83.0, 84.0]
[['Graders and Sorters, Agricultural Products'], '45-204', 1.0, 84.0, nan, 84.0, 84.0, 84.0, 84.0, 84.0]
[['Stock Clerks, Sales Floor', 'Marking Clerks', 'Order Fillers, Wholesale and Retail Sales', 'Stock Clerks- Stockroom, Warehouse, or Storage Yard'], '43-508', 4.0, 69.0, 11.343133018115703, 58.0, 61.75, 67.0, 74.25, 84.0]
[['Airline Pilots, Copilots, and Flight Engineers', 'Commercial Pilots'], '53-201', 2.0, 82.0, 1.4142135623730951, 81.0, 81.5, 82.0, 82.5, 83.0]
[['Brickmasons and Blockmasons', 'Stonemasons'], '47-202', 2.0, 81.5, 2.1213203435596424, 80.0, 80.75, 81.5, 82.25, 83.0]
[['Coroners', 'Licensing Examiners and Inspectors', 'Environmental Compliance Inspectors', 'Regulatory Affairs Specialists', 'Government 

[['Library Assistants, Clerical'], '43-412', 1.0, 73.0, nan, 73.0, 73.0, 73.0, 73.0, 73.0]
[['Mail Clerks and Mail Machine Operators, Except Postal Service'], '43-905', 1.0, 73.0, nan, 73.0, 73.0, 73.0, 73.0, 73.0]
[['Mechanical Door Repairers', 'Control and Valve Installers and Repairers, Except Mechanical Door'], '49-901', 2.0, 69.0, 5.656854249492381, 65.0, 67.0, 69.0, 71.0, 73.0]
[['Nuclear Power Reactor Operators', 'Power Distributors and Dispatchers', 'Power Plant Operators'], '51-801', 3.0, 63.333333333333336, 11.239810200058244, 51.0, 58.5, 66.0, 69.5, 73.0]
[['Park Naturalists', 'Soil and Water Conservationists', 'Range Managers', 'Foresters'], '19-103', 4.0, 52.5, 14.6401275039985, 41.0, 42.5, 48.0, 58.0, 73.0]
[['Parking Lot Attendants'], '53-602', 1.0, 73.0, nan, 73.0, 73.0, 73.0, 73.0, 73.0]
[['Pile-Driver Operators', 'Operating Engineers and Other Construction Equipment Operators', 'Paving, Surfacing, and Tamping Equipment Operators'], '47-207', 3.0, 61.666666666666664, 9

[['Audio-Visual and Multimedia Collections Specialists'], '25-901', 1.0, 59.0, nan, 59.0, 59.0, 59.0, 59.0, 59.0]
[['Bioinformatics Technicians', 'Statistical Assistants'], '43-911', 2.0, 52.0, 9.899494936611665, 45.0, 48.5, 52.0, 55.5, 59.0]
[['Continuous Mining Machine Operators', 'Mine Cutting and Channeling Machine Operators'], '47-504', 2.0, 56.0, 4.242640687119285, 53.0, 54.5, 56.0, 57.5, 59.0]
[['Electrical and Electronic Equipment Assemblers', 'Electromechanical Equipment Assemblers', 'Coil Winders, Tapers, and Finishers'], '51-202', 3.0, 58.666666666666664, 0.5773502691896258, 58.0, 58.5, 59.0, 59.0, 59.0]
[['Energy Auditors', 'Customs Brokers', 'Security Management Specialists', 'Business Continuity Planners', 'Online Merchants', 'Sustainability Specialists'], '13-119', 6.0, 51.5, 4.370354676682432, 47.0, 49.0, 50.0, 53.25, 59.0]
[['Insulation Workers, Mechanical', 'Insulation Workers, Floor, Ceiling, and Wall'], '47-213', 2.0, 58.5, 0.7071067811865476, 58.0, 58.25, 58.5, 58.

[['Emergency Management Directors'], '11-916', 1.0, 52.0, nan, 52.0, 52.0, 52.0, 52.0, 52.0]
[['Energy Brokers'], '41-309', 1.0, 52.0, nan, 52.0, 52.0, 52.0, 52.0, 52.0]
[['Food Scientists and Technologists', 'Animal Scientists', 'Soil and Plant Scientists'], '19-101', 3.0, 46.333333333333336, 5.507570547286102, 41.0, 43.5, 46.0, 49.0, 52.0]
[['Fraud Examiners, Investigators and Analysts', 'Financial Quantitative Analysts', 'Risk Management Specialists'], '13-209', 3.0, 47.0, 4.58257569495584, 43.0, 44.5, 46.0, 49.0, 52.0]
[['Insurance Policy Processing Clerks', 'Insurance Claims Clerks'], '43-904', 2.0, 49.0, 4.242640687119285, 46.0, 47.5, 49.0, 50.5, 52.0]
[['Insurance Underwriters', 'Financial Analysts', 'Personal Financial Advisors'], '13-205', 3.0, 45.333333333333336, 7.637626158259733, 37.0, 42.0, 47.0, 49.5, 52.0]
[['Librarians'], '25-402', 1.0, 52.0, nan, 52.0, 52.0, 52.0, 52.0, 52.0]
[['Machinists'], '51-404', 1.0, 52.0, nan, 52.0, 52.0, 52.0, 52.0, 52.0]
[['Production, Planni

[['Computer Operators'], '43-901', 1.0, 42.0, nan, 42.0, 42.0, 42.0, 42.0, 42.0]
[['Earth Drillers, Except Oil and Gas'], '47-502', 1.0, 42.0, nan, 42.0, 42.0, 42.0, 42.0, 42.0]
[['Social and Community Service Managers'], '11-915', 1.0, 42.0, nan, 42.0, 42.0, 42.0, 42.0, 42.0]
[['Computer and Information Systems Managers'], '11-302', 1.0, 41.0, nan, 41.0, 41.0, 41.0, 41.0, 41.0]
[['Human Resources Managers'], '11-312', 1.0, 41.0, nan, 41.0, 41.0, 41.0, 41.0, 41.0]
[['Management Analysts'], '13-111', 1.0, 41.0, nan, 41.0, 41.0, 41.0, 41.0, 41.0]
[['Operations Research Analysts'], '15-203', 1.0, 41.0, nan, 41.0, 41.0, 41.0, 41.0, 41.0]
[['Purchasing Managers'], '11-306', 1.0, 41.0, nan, 41.0, 41.0, 41.0, 41.0, 41.0]
[['Railroad Brake, Signal, and Switch Operators'], '53-402', 1.0, 41.0, nan, 41.0, 41.0, 41.0, 41.0, 41.0]
[['Mathematicians'], '15-202', 1.0, 40.0, nan, 40.0, 40.0, 40.0, 40.0, 40.0]
[['Remote Sensing Scientists and Technologists'], '19-209', 1.0, 40.0, nan, 40.0, 40.0, 40.0

{0: [['Choreographers', 'Dancers'],
  '27-203',
  2.0,
  99.0,
  1.4142135623730951,
  98.0,
  98.5,
  99.0,
  99.5,
  100.0],
 1: [['Dental Hygienists'],
  '29-202',
  1.0,
  100.0,
  nan,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0],
 2: [['Physical Therapists',
   'Radiation Therapists',
   'Occupational Therapists',
   'Exercise Physiologists',
   'Respiratory Therapists',
   'Music Therapists',
   'Low Vision Therapists, Orientation and Mobility Specialists, and Vision Rehabilitation Therapists',
   'Recreational Therapists',
   'Speech-Language Pathologists',
   'Art Therapists'],
  '29-112',
  10.0,
  88.9,
  8.5693251387337,
  73.0,
  84.5,
  90.5,
  94.5,
  100.0],
 3: [['Sports Medicine Physicians',
   'Urologists',
   'Dermatologists',
   'Obstetricians and Gynecologists',
   'Surgeons',
   'Anesthesiologists',
   'Hospitalists',
   'Ophthalmologists',
   'Family and General Practitioners',
   'Physical Medicine and Rehabilitation Physicians',
   'Internists, General',
   'P

In [6]:
#then we build a dataframe from this dictionary
columnlist=["jobs","code"]
columnlist.extend((contactwith_others["Context"].describe()).index.tolist())
dataframe=pd.DataFrame().from_dict(groudict).transpose()
dataframe.columns=columnlist
dataframe=dataframe.fillna(0)
#and add the relative std as a column
dataframe["rel_std"]=dataframe["std"]/dataframe["mean"]
dataframe=dataframe.sort_values(by="rel_std")
#turn out, on average the grading of physical proximity only differs by about 7 per cent
dataframe["rel_std"].mean()

0.07262895229881285

So now we have grouped job of the same broader job category and calculated the mean physical proximity for these jobs. And we write the datafile out to be now classified by German job codes.

In [7]:
#and we write this list out 
dataframe.to_csv("sum_up_of_us_codes.csv",index=False)

# Coding US to German job codes

So, next up was the tidious part. I had to classify 438 job groups by hand using [the official job encoding list of the German labour ministry](https://www.klassifikationsserver.de/klassService/jsp/common/url.jsf?variant=kldb2010). I always added to each US-job group the three digit job classification in Germany that I deemed to be fitting. (Saved under "jobs_with_context_classified.csv"). However, to check this classification, I also built a simple scraper to do the same job that I did. I used the aforementioned csv, ran it through deepl to have a German translation of all jobs. With this German translation of the jobs, I have sent a query for each job group [to the Berufenet database](https://berufenet.arbeitsagentur.de/berufenet/faces/index?path=null) of the German labour ministry to check which job code would be the result of such a query. The used scraper is saved under "Puppeteer_Arbeitsagentur", I ended up with three csvs that I got as a result of different forms of query with this scraper (sending the pure translation to the website, split at "hyphen" and only send the first word, split at " " space and send only the last word). Then I merged these three csvs by hand into one and cleaned it up as follows.

In [8]:
raw_data=pd.read_csv("../RawData/halukaschreibt_zusammen.csv")
dropped=raw_data

In [9]:
# we create one master column for the jobcode number and fill it first nans
dropped['number_zusammen'] = np.nan
#if there was a result retrieved from the scraper, we put the number from this query in the master column, if not we leave it as it is
dropped['number_zusammen'] = np.where(dropped['Resultatsberuf'].isna(), np.nan,dropped['Nummer'])
#and we do this for all the three types of columns we had
dropped['number_zusammen'] = np.where(dropped['Resultatsberuf2'].isna(), dropped['number_zusammen'],dropped['Nummer2'])
dropped['number_zusammen'] = np.where(dropped['Resultatsberuf_3'].isna(), dropped['number_zusammen'],dropped['Nummer_3'])
#and also with a master column for the job name
dropped['Resultatsberuf_zusammen'] = np.nan
dropped['Resultatsberuf_zusammen'] = np.where(dropped['Resultatsberuf'].isna(), np.nan,dropped['Resultatsberuf'])
dropped['Resultatsberuf_zusammen'] = np.where(dropped['Resultatsberuf2'].isna(), dropped['Resultatsberuf_zusammen'],dropped['Resultatsberuf2'])
dropped['Resultatsberuf_zusammen'] = np.where(dropped['Resultatsberuf_3'].isna(), dropped['Resultatsberuf_zusammen'],dropped['Resultatsberuf_3'])

#and so get a reduced dataframe
summed_scraper=dropped[["Berufsübersetzung","number_zusammen","Resultatsberuf_zusammen"]]
summed_scraper.to_csv("scraper_results.csv",index=False)
summed_scraper

,Berufsübersetzung,number_zusammen,Resultatsberuf_zusammen
0,Dentalhygieni,81113-10,Dentalhygieniker/in
1,Choreograp,94224-10,Ballettmeister/in
2,Kieferorthopä,81113-10,Kieferorthopädische/r Fachhelfer/in
3,Krankenschwes,82182-10,Ambulante/r Pfleger/in
4,Medizinische Notfalltechniker und Rettungssanitä,81342-10,Rettungssanitäter/in
...,...,...,...
434,Erdölingenie,21114-11,Ingenieur/in - Rohstoffgewinnung und -aufberei...
435,Ausschreibungen für Brücken und Schleus,NaN,NaN
436,Log Grader und Scal,NaN,NaN
437,Jäger und Fallenstel,NaN,NaN


Also, I merged the by hand classified data with the data from the scraper and threw out all the jobs where neither the scraper nor I found a German job code.

In [10]:
hand_classified=pd.read_csv("jobs_with_context_classified.csv",dtype={"germancode":str})
hand_scraper=pd.concat([hand_classified,summed_scraper],axis=1)
hand_scraper["three_letter_scraper"]=hand_scraper["number_zusammen"].str[0:3]
hand_scraper=hand_scraper.dropna(subset=["germancode","three_letter_scraper"],how="all")
hand_scraper[hand_scraper["germancode"]==hand_scraper["three_letter_scraper"]]

,germancode,jobs,code,count,mean,std,min,25%,50%,75%,max,rel_std,Berufsübersetzung,number_zusammen,Resultatsberuf_zusammen,three_letter_scraper
1,942,"['Choreographers', 'Dancers']",27-203,2.0,99.00,1.414214,98.0,98.50,99.0,99.50,100.0,0.014285,Choreograp,94224-10,Ballettmeister/in,942
4,813,['Emergency Medical Technicians and Paramedics'],29-204,1.0,97.00,0.000000,97.0,97.00,97.0,97.00,97.0,0.000000,Medizinische Notfalltechniker und Rettungssanitä,81342-10,Rettungssanitäter/in,813
6,514,['Flight Attendants'],53-203,1.0,96.00,0.000000,96.0,96.00,96.0,96.00,96.0,0.000000,Flugbeglei,51422-10,Flugbegleiter/in,514
8,817,"['Physical Therapist Aides', 'Physical Therapi...",31-202,2.0,94.50,0.707107,94.0,94.25,94.5,94.75,95.0,0.007483,Physiotherapeu,81713-10,Physiotherapeut/in,817
9,823,"['Barbers', 'Hairdressers, Hairstylists, and C...",39-501,2.0,93.50,2.121320,92.0,92.75,93.5,94.25,95.0,0.022688,Barbi,82312-10,Herrenfriseur/in,823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,924,"['Technical Writers', 'Copy Writers', 'Poets, ...",27-304,4.0,39.75,17.858238,14.0,35.00,45.5,50.25,54.0,0.449264,Technische Redakte,92413-10,Redakteur/in,924
413,414,"['Astronomers', 'Physicists']",19-201,2.0,39.00,11.313709,31.0,35.00,39.0,43.00,47.0,0.290095,Astrono,41484-11,"Astrophysiker/in, Astronom/in",414
416,922,['Public Relations and Fundraising Managers'],11-203,1.0,38.00,0.000000,38.0,38.00,38.0,38.00,38.0,0.000000,Manager für Öffentlichkeitsarbeit und Fundrais,92203-10,Fundraiser/in,922
423,611,['Sales Engineers'],41-903,1.0,36.00,0.000000,36.0,36.00,36.0,36.00,36.0,0.000000,Vertriebsingenie,61124-10,Vertriebsingenieur/in,611


And also, I marked whether the result from the scraper and my classification were matching. If this was the case then the checked column was marked with a "y". Otherwise it was left empty.

In [11]:
hand_scraper["checked"]=""
hand_scraper['checked'] = np.where(hand_scraper["germancode"].str[:2]==hand_scraper["three_letter_scraper"].str[:2], "y",hand_scraper["checked"])
hand_scraper.to_csv("hand_scraper.csv", index=False)
hand_scraper

,germancode,jobs,code,count,mean,std,min,25%,50%,75%,max,rel_std,Berufsübersetzung,number_zusammen,Resultatsberuf_zusammen,three_letter_scraper,checked
0,814,['Dental Hygienists'],29-202,1.0,100.0,0.000000,100.0,100.0,100.0,100.0,100.0,0.000000,Dentalhygieni,81113-10,Dentalhygieniker/in,811,y
1,942,"['Choreographers', 'Dancers']",27-203,2.0,99.0,1.414214,98.0,98.5,99.0,99.5,100.0,0.014285,Choreograp,94224-10,Ballettmeister/in,942,y
2,814,"['Orthodontists', 'Oral and Maxillofacial Surg...",29-102,4.0,97.0,3.366502,92.0,96.5,98.5,99.0,99.0,0.034706,Kieferorthopä,81113-10,Kieferorthopädische/r Fachhelfer/in,811,y
3,813,['Nurse Midwives'],29-116,1.0,97.0,0.000000,97.0,97.0,97.0,97.0,97.0,0.000000,Krankenschwes,82182-10,Ambulante/r Pfleger/in,821,
4,813,['Emergency Medical Technicians and Paramedics'],29-204,1.0,97.0,0.000000,97.0,97.0,97.0,97.0,97.0,0.000000,Medizinische Notfalltechniker und Rettungssanitä,81342-10,Rettungssanitäter/in,813,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,722,"['Compensation, Benefits, and Job Analysis Spe...",13-114,1.0,32.0,0.000000,32.0,32.0,32.0,32.0,32.0,0.000000,Spezialisten für Entlohn,NaN,NaN,NaN,
432,914,"['Environmental Economists', 'Economists']",19-301,2.0,31.0,2.828427,29.0,30.0,31.0,32.0,33.0,0.091240,Umweltökono,42314-11,Betriebswirt/in (Hochschule) - Umweltökonomie,423,
434,412,['Petroleum Engineers'],17-217,1.0,30.0,0.000000,30.0,30.0,30.0,30.0,30.0,0.000000,Erdölingenie,21114-11,Ingenieur/in - Rohstoffgewinnung und -aufberei...,211,
436,117,"['Log Graders and Scalers', 'Logging Equipment...",45-402,3.0,23.0,21.931712,7.0,10.5,14.0,31.0,48.0,0.953553,Log Grader und Scal,NaN,NaN,NaN,


I took this file then again and for all the round about 300 job groups where a difference between the results of the two methods existed, I checked again the webpage of Berufenet to find the fitting classification. If this second check 
was the same three digit number as with the first run, I also inserted a "y", otherwise I inserted the number.

# Evaluation of the classification

So how good is the speeded up version of intracoder reliability for the classification? Well, in over 60 percent of the cases I or the scraper reclassified the jobs in exactly the same number code as previously.

In [12]:
data=pd.read_csv("jobcodes_evaluated_part_2.csv",dtype={"germancode":str,"three_letter_scraper":str})
data

,germancode,checked,Berufsübersetzung,jobs,code,count,mean,std,min,25%,50%,75%,max,rel_std,Berufsübersetzung.1,number_zusammen,Resultatsberuf_zusammen,three_letter_scraper,matched
0,813,y,Krankenschwes,['Nurse Midwives'],29-116,1.0,97.000000,0.000000,97.0,97.00,97.0,97.00,97.0,0.000000,Krankenschwes,82182-10,Ambulante/r Pfleger/in,821,False
1,817,811,Fußspezialis,['Podiatrists'],29-108,1.0,95.000000,0.000000,95.0,95.00,95.0,95.00,95.0,0.000000,Fußspezialis,NaN,NaN,NaN,False
2,514,y,Transportbeglei,"['Transportation Attendants, Except Flight Att...",53-606,1.0,93.000000,0.000000,93.0,93.00,93.0,93.00,93.0,0.000000,Transportbeglei,NaN,NaN,NaN,False
3,825,814,Orthopäden und Protheti,"['Orthotists and Prosthetists', 'Neurodiagnost...",29-209,6.0,91.333333,6.801961,79.0,89.50,94.0,95.50,97.0,0.074474,Orthopäden und Protheti,NaN,NaN,NaN,False
4,813,832,Haushaltshil,"['Home Health Aides', 'Nursing Assistants', 'O...",31-101,4.0,90.250000,5.377422,84.0,87.75,90.0,92.50,97.0,0.059584,Haushaltshil,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,924,y,Technische Redakte,"['Technical Writers', 'Copy Writers', 'Poets, ...",27-304,4.0,39.750000,17.858238,14.0,35.00,45.5,50.25,54.0,0.449264,Technische Redakte,92413-10,Redakteur/in,924,True
417,414,y,Astrono,"['Astronomers', 'Physicists']",19-201,2.0,39.000000,11.313709,31.0,35.00,39.0,43.00,47.0,0.290095,Astrono,41484-11,"Astrophysiker/in, Astronom/in",414,True
418,922,y,Manager für Öffentlichkeitsarbeit und Fundrais,['Public Relations and Fundraising Managers'],11-203,1.0,38.000000,0.000000,38.0,38.00,38.0,38.00,38.0,0.000000,Manager für Öffentlichkeitsarbeit und Fundrais,92203-10,Fundraiser/in,922,True
419,611,y,Vertriebsingenie,['Sales Engineers'],41-903,1.0,36.000000,0.000000,36.0,36.00,36.0,36.00,36.0,0.000000,Vertriebsingenie,61124-10,Vertriebsingenieur/in,611,True


In [13]:
len(data[data["checked"]=="y"])/len(data)

0.6128266033254157

And if it is about whether at least the first number which stands for the working sector matched, we get an additional 19 percent, so round about 80 percent of the jobs were classified correctly. This is not great but something we can work with.

In [14]:
len(data[data["germancode"].str[:1]==data["checked"].str[:1]])/len(data)

0.19002375296912113

# Actual analysis
So we do the same analysis in three ways to be sure that faulty classification cannot be the reason why an analysis shows a signal or not. But first we load in German labour data regarding the median wage

In [15]:
german_jobs=pd.read_csv("../RawData/jobs_d.csv", skiprows=6)
german_jobs=german_jobs[["Unnamed: 0","Median"]]
german_jobs["codes"]=german_jobs["Unnamed: 0"].str[:3]
german_jobs=german_jobs.rename(columns={"Unnamed: 0":"Bezeichnung"})
german_jobs["Median"]=german_jobs["Median"].str.replace("\(","")
german_jobs["Median"]=german_jobs["Median"].str.replace("\)","")
german_jobs=german_jobs.dropna()
german_jobs

,Bezeichnung,Median,codes
1,111 Landwirtschaft,9.99,111
2,112 Tierwirtschaft,9.59,112
3,113 Pferdewirtschaft,8.84,113
4,114 Fischwirtschaft,10.00,114
5,115 Tierpflege,10.28,115
...,...,...,...
140,947 Museumstechnik und -management,19.21,947
141,011 Offiziere,25.23,011
142,012 Unteroffiziere mit Portepee,17.90,012
143,013 Unteroffiziere ohne Portepee,14.06,013


## Clear Matches
This is the dataset where we only use data where the first run of classification and the second run of classification led to the exact same results. We merge it with the data from the Ministry of Labour regarding wages.

In [16]:
clear_matches=data

clear_matches['number_to_merge'] =np.nan
clear_matches['number_to_merge'] = np.where(clear_matches['checked']=="y",clear_matches['germancode'] ,clear_matches['number_to_merge'] )
clear_matches

,germancode,checked,Berufsübersetzung,jobs,code,count,mean,std,min,25%,50%,75%,max,rel_std,Berufsübersetzung.1,number_zusammen,Resultatsberuf_zusammen,three_letter_scraper,matched,number_to_merge
0,813,y,Krankenschwes,['Nurse Midwives'],29-116,1.0,97.000000,0.000000,97.0,97.00,97.0,97.00,97.0,0.000000,Krankenschwes,82182-10,Ambulante/r Pfleger/in,821,False,813
1,817,811,Fußspezialis,['Podiatrists'],29-108,1.0,95.000000,0.000000,95.0,95.00,95.0,95.00,95.0,0.000000,Fußspezialis,NaN,NaN,NaN,False,NaN
2,514,y,Transportbeglei,"['Transportation Attendants, Except Flight Att...",53-606,1.0,93.000000,0.000000,93.0,93.00,93.0,93.00,93.0,0.000000,Transportbeglei,NaN,NaN,NaN,False,514
3,825,814,Orthopäden und Protheti,"['Orthotists and Prosthetists', 'Neurodiagnost...",29-209,6.0,91.333333,6.801961,79.0,89.50,94.0,95.50,97.0,0.074474,Orthopäden und Protheti,NaN,NaN,NaN,False,NaN
4,813,832,Haushaltshil,"['Home Health Aides', 'Nursing Assistants', 'O...",31-101,4.0,90.250000,5.377422,84.0,87.75,90.0,92.50,97.0,0.059584,Haushaltshil,NaN,NaN,NaN,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,924,y,Technische Redakte,"['Technical Writers', 'Copy Writers', 'Poets, ...",27-304,4.0,39.750000,17.858238,14.0,35.00,45.5,50.25,54.0,0.449264,Technische Redakte,92413-10,Redakteur/in,924,True,924
417,414,y,Astrono,"['Astronomers', 'Physicists']",19-201,2.0,39.000000,11.313709,31.0,35.00,39.0,43.00,47.0,0.290095,Astrono,41484-11,"Astrophysiker/in, Astronom/in",414,True,414
418,922,y,Manager für Öffentlichkeitsarbeit und Fundrais,['Public Relations and Fundraising Managers'],11-203,1.0,38.000000,0.000000,38.0,38.00,38.0,38.00,38.0,0.000000,Manager für Öffentlichkeitsarbeit und Fundrais,92203-10,Fundraiser/in,922,True,922
419,611,y,Vertriebsingenie,['Sales Engineers'],41-903,1.0,36.000000,0.000000,36.0,36.00,36.0,36.00,36.0,0.000000,Vertriebsingenie,61124-10,Vertriebsingenieur/in,611,True,611


In [17]:
merged_clear_matches=pd.merge(german_jobs,clear_matches,left_on="codes",right_on="number_to_merge")
merged_clear_matches.to_csv("german_to_us_jobs_clear_matches.csv", index=False)
merged_clear_matches=merged_clear_matches[["Bezeichnung","jobs","Median","mean","number_to_merge"]]
merged_clear_matches["Median"]=pd.to_numeric(merged_clear_matches["Median"],errors="coerce")
merged_clear_matches["mean"]=pd.to_numeric(merged_clear_matches["mean"])
merged_clear_matches=merged_clear_matches.dropna()
merged_clear_matches=merged_clear_matches.rename(columns={"Median":"Lohn","mean":"Nähe"})
merged_clear_matches["Sektor"]=merged_clear_matches["number_to_merge"].str[:1]
merged_clear_matches["Bezeichnung_Jobs"]=merged_clear_matches["Bezeichnung"]+merged_clear_matches["jobs"]

merged_clear_matches

,Bezeichnung,jobs,Lohn,Nähe,number_to_merge,Sektor,Bezeichnung_Jobs
0,111 Landwirtschaft,"['Graders and Sorters, Agricultural Products']",9.99,84.000000,111,1,"111 Landwirtschaft['Graders and Sorters, Agric..."
1,111 Landwirtschaft,['Farm and Home Management Advisors'],9.99,43.000000,111,1,111 Landwirtschaft['Farm and Home Management A...
2,111 Landwirtschaft,"['Agricultural Equipment Operators', 'Farmwork...",9.99,41.500000,111,1,111 Landwirtschaft['Agricultural Equipment Ope...
3,111 Landwirtschaft,['Agricultural Inspectors'],9.99,68.000000,111,1,111 Landwirtschaft['Agricultural Inspectors']
4,111 Landwirtschaft,"['Farm and Ranch Managers', 'Aquacultural Mana...",9.99,55.666667,111,1,"111 Landwirtschaft['Farm and Ranch Managers', ..."
...,...,...,...,...,...,...,...
252,935 Kunsthandwerkliche Metallgestaltung,"['Gem and Diamond Workers', 'Jewelers', 'Preci...",13.42,54.333333,935,9,935 Kunsthandwerkliche Metallgestaltung['Gem a...
253,"941 Musik-, Gesang-, Dirigententätigkeiten","['Singers', 'Music Directors', 'Musicians, Ins...",24.93,67.250000,941,9,"941 Musik-, Gesang-, Dirigententätigkeiten['Si..."
255,943 Moderation und Unterhaltung,"['Gaming and Sports Book Writers and Runners',...",19.02,86.500000,943,9,943 Moderation und Unterhaltung['Gaming and Sp...
256,"945 Veranstaltungs-, Kamera-, Tontechnik","['Camera Operators, Television, Video, and Mot...",17.61,58.500000,945,9,"945 Veranstaltungs-, Kamera-, Tontechnik['Came..."


## 1st run classification
This is the dataset where we use data where the first run of classification and the second run of classification led to the exact same results *and* we use the classification of the first run, if first and second run at least idenfitfied the same working sector (first digit)

In [18]:
classified_1strun=data
classified_1strun['number_to_merge'] =np.nan
classified_1strun['number_to_merge'] = np.where(classified_1strun['checked']=="y",classified_1strun['germancode'] ,classified_1strun['number_to_merge'] )
classified_1strun['number_to_merge'] = np.where(classified_1strun["germancode"].str[:1]==classified_1strun["checked"].str[:1],classified_1strun['germancode'] ,classified_1strun['number_to_merge'] )
classified_1strun

,germancode,checked,Berufsübersetzung,jobs,code,count,mean,std,min,25%,50%,75%,max,rel_std,Berufsübersetzung.1,number_zusammen,Resultatsberuf_zusammen,three_letter_scraper,matched,number_to_merge
0,813,y,Krankenschwes,['Nurse Midwives'],29-116,1.0,97.000000,0.000000,97.0,97.00,97.0,97.00,97.0,0.000000,Krankenschwes,82182-10,Ambulante/r Pfleger/in,821,False,813
1,817,811,Fußspezialis,['Podiatrists'],29-108,1.0,95.000000,0.000000,95.0,95.00,95.0,95.00,95.0,0.000000,Fußspezialis,NaN,NaN,NaN,False,817
2,514,y,Transportbeglei,"['Transportation Attendants, Except Flight Att...",53-606,1.0,93.000000,0.000000,93.0,93.00,93.0,93.00,93.0,0.000000,Transportbeglei,NaN,NaN,NaN,False,514
3,825,814,Orthopäden und Protheti,"['Orthotists and Prosthetists', 'Neurodiagnost...",29-209,6.0,91.333333,6.801961,79.0,89.50,94.0,95.50,97.0,0.074474,Orthopäden und Protheti,NaN,NaN,NaN,False,825
4,813,832,Haushaltshil,"['Home Health Aides', 'Nursing Assistants', 'O...",31-101,4.0,90.250000,5.377422,84.0,87.75,90.0,92.50,97.0,0.059584,Haushaltshil,NaN,NaN,NaN,False,813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,924,y,Technische Redakte,"['Technical Writers', 'Copy Writers', 'Poets, ...",27-304,4.0,39.750000,17.858238,14.0,35.00,45.5,50.25,54.0,0.449264,Technische Redakte,92413-10,Redakteur/in,924,True,924
417,414,y,Astrono,"['Astronomers', 'Physicists']",19-201,2.0,39.000000,11.313709,31.0,35.00,39.0,43.00,47.0,0.290095,Astrono,41484-11,"Astrophysiker/in, Astronom/in",414,True,414
418,922,y,Manager für Öffentlichkeitsarbeit und Fundrais,['Public Relations and Fundraising Managers'],11-203,1.0,38.000000,0.000000,38.0,38.00,38.0,38.00,38.0,0.000000,Manager für Öffentlichkeitsarbeit und Fundrais,92203-10,Fundraiser/in,922,True,922
419,611,y,Vertriebsingenie,['Sales Engineers'],41-903,1.0,36.000000,0.000000,36.0,36.00,36.0,36.00,36.0,0.000000,Vertriebsingenie,61124-10,Vertriebsingenieur/in,611,True,611


In [19]:
merged_classified_1strun=pd.merge(german_jobs,classified_1strun,left_on="codes",right_on="number_to_merge")
merged_classified_1strun.to_csv("german_to_us_jobs_1strun.csv", index=False)
merged_classified_1strun=merged_classified_1strun[["Bezeichnung","jobs","Median","mean","number_to_merge"]]
merged_classified_1strun["Median"]=pd.to_numeric(merged_classified_1strun["Median"],errors="coerce")
merged_classified_1strun["mean"]=pd.to_numeric(merged_classified_1strun["mean"])
merged_classified_1strun=merged_classified_1strun.dropna()
merged_classified_1strun=merged_classified_1strun.rename(columns={"Median":"Lohn","mean":"Nähe"})
merged_classified_1strun["Sektor"]=merged_classified_1strun["number_to_merge"].str[:1]
merged_classified_1strun["Bezeichnung_Jobs"]=merged_classified_1strun["Bezeichnung"]+merged_classified_1strun["jobs"]

merged_classified_1strun

,Bezeichnung,jobs,Lohn,Nähe,number_to_merge,Sektor,Bezeichnung_Jobs
0,111 Landwirtschaft,"['Graders and Sorters, Agricultural Products']",9.99,84.00,111,1,"111 Landwirtschaft['Graders and Sorters, Agric..."
1,111 Landwirtschaft,['First-Line Supervisors of Animal Husbandry a...,9.99,45.50,111,1,111 Landwirtschaft['First-Line Supervisors of ...
2,111 Landwirtschaft,['Farm and Home Management Advisors'],9.99,43.00,111,1,111 Landwirtschaft['Farm and Home Management A...
3,111 Landwirtschaft,"['Agricultural Equipment Operators', 'Farmwork...",9.99,41.50,111,1,111 Landwirtschaft['Agricultural Equipment Ope...
4,111 Landwirtschaft,['Agricultural Inspectors'],9.99,68.00,111,1,111 Landwirtschaft['Agricultural Inspectors']
...,...,...,...,...,...,...,...
332,"941 Musik-, Gesang-, Dirigententätigkeiten","['Singers', 'Music Directors', 'Musicians, Ins...",24.93,67.25,941,9,"941 Musik-, Gesang-, Dirigententätigkeiten['Si..."
334,943 Moderation und Unterhaltung,"['Gaming and Sports Book Writers and Runners',...",19.02,86.50,943,9,943 Moderation und Unterhaltung['Gaming and Sp...
335,"944 Theater-, Film- und Fernsehproduktion","['Talent Directors', 'Directors- Stage, Motion...",18.95,63.00,944,9,"944 Theater-, Film- und Fernsehproduktion['Tal..."
336,"945 Veranstaltungs-, Kamera-, Tontechnik","['Camera Operators, Television, Video, and Mot...",17.61,58.50,945,9,"945 Veranstaltungs-, Kamera-, Tontechnik['Came..."


## 2nd run classification
This is the dataset where we use data where the first run of classification and the second run of classification led to the exact same results *and* we use the classification of the second run, if first and second run at least idenfitfied the same working sector (first digit)

In [20]:
classified_2ndrun=data
classified_2ndrun['number_to_merge'] =np.nan
classified_2ndrun['number_to_merge'] = np.where(classified_2ndrun['checked']=="y",classified_2ndrun['germancode'] ,classified_2ndrun['number_to_merge'] )
classified_2ndrun['number_to_merge'] = np.where(classified_2ndrun["germancode"].str[:1]==classified_2ndrun["checked"].str[:1],classified_2ndrun['checked'] ,classified_2ndrun['number_to_merge'] )
classified_2ndrun

,germancode,checked,Berufsübersetzung,jobs,code,count,mean,std,min,25%,50%,75%,max,rel_std,Berufsübersetzung.1,number_zusammen,Resultatsberuf_zusammen,three_letter_scraper,matched,number_to_merge
0,813,y,Krankenschwes,['Nurse Midwives'],29-116,1.0,97.000000,0.000000,97.0,97.00,97.0,97.00,97.0,0.000000,Krankenschwes,82182-10,Ambulante/r Pfleger/in,821,False,813
1,817,811,Fußspezialis,['Podiatrists'],29-108,1.0,95.000000,0.000000,95.0,95.00,95.0,95.00,95.0,0.000000,Fußspezialis,NaN,NaN,NaN,False,811
2,514,y,Transportbeglei,"['Transportation Attendants, Except Flight Att...",53-606,1.0,93.000000,0.000000,93.0,93.00,93.0,93.00,93.0,0.000000,Transportbeglei,NaN,NaN,NaN,False,514
3,825,814,Orthopäden und Protheti,"['Orthotists and Prosthetists', 'Neurodiagnost...",29-209,6.0,91.333333,6.801961,79.0,89.50,94.0,95.50,97.0,0.074474,Orthopäden und Protheti,NaN,NaN,NaN,False,814
4,813,832,Haushaltshil,"['Home Health Aides', 'Nursing Assistants', 'O...",31-101,4.0,90.250000,5.377422,84.0,87.75,90.0,92.50,97.0,0.059584,Haushaltshil,NaN,NaN,NaN,False,832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,924,y,Technische Redakte,"['Technical Writers', 'Copy Writers', 'Poets, ...",27-304,4.0,39.750000,17.858238,14.0,35.00,45.5,50.25,54.0,0.449264,Technische Redakte,92413-10,Redakteur/in,924,True,924
417,414,y,Astrono,"['Astronomers', 'Physicists']",19-201,2.0,39.000000,11.313709,31.0,35.00,39.0,43.00,47.0,0.290095,Astrono,41484-11,"Astrophysiker/in, Astronom/in",414,True,414
418,922,y,Manager für Öffentlichkeitsarbeit und Fundrais,['Public Relations and Fundraising Managers'],11-203,1.0,38.000000,0.000000,38.0,38.00,38.0,38.00,38.0,0.000000,Manager für Öffentlichkeitsarbeit und Fundrais,92203-10,Fundraiser/in,922,True,922
419,611,y,Vertriebsingenie,['Sales Engineers'],41-903,1.0,36.000000,0.000000,36.0,36.00,36.0,36.00,36.0,0.000000,Vertriebsingenie,61124-10,Vertriebsingenieur/in,611,True,611


Here we connect the data to the German statistics on wages and do some data cleaning

In [21]:
#now we merge this with results for wages from Germany
merged_classified_2ndrun=pd.merge(german_jobs,classified_2ndrun,left_on="codes",right_on="number_to_merge")
merged_classified_2ndrun.to_csv("german_to_us_jobs_2ndrun.csv", index=False)

#amd we reduce it to only the most important columns and convert them to numerics
merged_classified_2ndrun=merged_classified_2ndrun[["Bezeichnung","jobs","Median","mean","number_to_merge"]]
merged_classified_2ndrun["Median"]=pd.to_numeric(merged_classified_2ndrun["Median"],errors="coerce")
merged_classified_2ndrun["mean"]=pd.to_numeric(merged_classified_2ndrun["mean"])

#then we drop all results with nas
merged_classified_2ndrun=merged_classified_2ndrun.dropna()

#rename the columns
merged_classified_2ndrun=merged_classified_2ndrun.rename(columns={"Median":"Lohn","mean":"Nähe"})

#and read out the job sector
merged_classified_2ndrun["Sektor"]=merged_classified_2ndrun["number_to_merge"].str[:1]
merged_classified_2ndrun["Bezeichnung_Jobs"]=merged_classified_2ndrun["Bezeichnung"]+merged_classified_2ndrun["jobs"]

merged_classified_2ndrun.sort_values(by="Nähe")

,Bezeichnung,jobs,Lohn,Nähe,number_to_merge,Sektor,Bezeichnung_Jobs
265,814 Human- und Zahnmedizin,"['Orthotists and Prosthetists', 'Neurodiagnost...",36.96,91.333333,814,8,814 Human- und Zahnmedizin['Orthotists and Pro...
266,814 Human- und Zahnmedizin,['Dental Hygienists'],36.96,100.000000,814,8,814 Human- und Zahnmedizin['Dental Hygienists']
267,814 Human- und Zahnmedizin,"['Orthodontists', 'Oral and Maxillofacial Surg...",36.96,97.000000,814,8,"814 Human- und Zahnmedizin['Orthodontists', 'O..."
268,814 Human- und Zahnmedizin,"['Anesthesiologists', 'Hospitalists', 'Nuclear...",36.96,83.052632,814,8,814 Human- und Zahnmedizin['Anesthesiologists'...


# Correlation analysis and visualisation
Now we wanna check for R^2 and for the p-value for the several worksectors and for the different classifications. We first define the specs of the visualization.

In [23]:
charts=alt.Chart().mark_point().encode(
    x='Nähe',
    y='Lohn',
     tooltip="Bezeichnung_Jobs",
    color="Bezeichnung"
)
bakedchart=charts + charts.transform_regression('Nähe','Lohn').mark_line()


And now we run this analysis for clear matches, the 1st classification run and 2nd classification run

In [24]:
dataset_to_be_used=merged_clear_matches
sectorlist=dataset_to_be_used["Sektor"].unique().tolist()
sectorlist.sort()
for i in (sectorlist):
    print(i)
    subset=dataset_to_be_used[dataset_to_be_used["Sektor"]==i]
    subset=subset[subset["number_to_merge"]!="814"]
    result=(scipy.stats.linregress(subset["Nähe"],subset["Lohn"]))
    print(result.pvalue)
    print(result.rvalue**2)

    combined = alt.layer(charts + charts.transform_regression('Nähe','Lohn').mark_line(), data=subset)
    combined.display()

1
0.018063472104251646
0.38416987062246083


alt.LayerChart(...)

2
0.38070903965501923
0.015402609664760014


alt.LayerChart(...)

3
0.10151899374922574
0.14987794560374476


...

alt.LayerChart(...)

8
2.9583929454043816e-06
0.39460395554777283


alt.LayerChart(...)

9
0.9703611199377895
0.00011034765028729514


alt.LayerChart(...)

In [25]:
dataset_to_be_used=merged_classified_1strun
sectorlist=dataset_to_be_used["Sektor"].unique().tolist()
sectorlist.sort()
for i in (sectorlist):
    print(i)
    subset=dataset_to_be_used[dataset_to_be_used["Sektor"]==i]
    subset=subset[subset["number_to_merge"]!="814"]

    result=(scipy.stats.linregress(subset["Lohn"], subset["Nähe"]))
    print(result.pvalue)
    print(result.rvalue**2)

    combined = alt.layer(charts + charts.transform_regression('Nähe','Lohn').mark_line(), data=subset)
    combined.display()

1
0.26621752598380083
0.08165237235384798


alt.LayerChart(...)

2
0.9201937036034954
0.00013655867352307605


alt.LayerChart(...)

3
0.03952494401113892
0.19520667960921845


...

alt.LayerChart(...)

8
1.6920967422041883e-09
0.4510368727332692


alt.LayerChart(...)

9
0.7639083216601454
0.005800107007528869


alt.LayerChart(...)

In [26]:
dataset_to_be_used=merged_classified_2ndrun
sectorlist=dataset_to_be_used["Sektor"].unique().tolist()
sectorlist.sort()
for i in (sectorlist):
    print(i)
    subset=dataset_to_be_used[dataset_to_be_used["Sektor"]==i]
    subset=subset[subset["number_to_merge"]!="814"]

    result=(scipy.stats.linregress(subset["Lohn"], subset["Nähe"]))
    print(result.pvalue)
    print(result.rvalue**2)

    combined = alt.layer(charts + charts.transform_regression('Nähe','Lohn').mark_line(), data=subset)
    combined.display()

1
0.14546139408089603
0.13584650867581421


alt.LayerChart(...)

2
0.9869487995802184
3.640502659299374e-06


alt.LayerChart(...)

3
0.040915615778079034
0.19278527426866285


...

alt.LayerChart(...)

8
4.243309498155021e-08
0.3962489284646368


alt.LayerChart(...)

9
0.9588496349298119
0.0001834793024556954


alt.LayerChart(...)

# Preliminary result

For job sector 6 (Trade, Tourism, Gastronomy) and 8 (Social and Health), these correlations are significant and our r-square is big. So we seem to have an important correlation here.

# Splitting up results for Germany, Berlin and Brandenburg
To turn this into a story for rbb, we now also used the data for the different monthly salaries of these jobs. Again all results stay significant

In [27]:
base=pd.read_excel("../RawData/monatsentgelte.xlsx", sheet_name="Deutschland")
base["3dig_code"]=base["Berufsgruppe"].str[:3]
base["Insgesamt"]=pd.to_numeric(base["Insgesamt"]*1000,errors="coerce")
base=base.dropna(subset=["Insgesamt"],axis=0)
base=base[["3dig_code","Insgesamt"]]
for i in ["Berlin","Brandenburg"]:
    data=pd.read_excel("../RawData/monatsentgelte.xlsx", sheet_name=i)
    data["3dig_code"]=data["Berufsgruppe"].str[:3]
    data["Insgesamt"]=pd.to_numeric(data["Insgesamt"]*1000,errors="coerce")
    data=data.dropna(subset=["Insgesamt"],axis=0)
    data=data[["3dig_code","Insgesamt"]]
    base=pd.merge(base,data,on="3dig_code",how="outer")
base=base.rename(columns={"Insgesamt_x":"Deutschland","Insgesamt_y":"Berlin","Insgesamt":"Brandenburg"})

In [28]:
test=pd.merge(merged_classified_2ndrun,base,left_on="number_to_merge",right_on="3dig_code")
test

,Bezeichnung,jobs,Lohn,Nähe,number_to_merge,Sektor,Bezeichnung_Jobs,3dig_code,Deutschland,Berlin,Brandenburg
0,111 Landwirtschaft,"['Graders and Sorters, Agricultural Products']",9.99,84.000000,111,1,"111 Landwirtschaft['Graders and Sorters, Agric...",111,2161.0,NaN,1933.0
1,111 Landwirtschaft,['Farm and Home Management Advisors'],9.99,43.000000,111,1,111 Landwirtschaft['Farm and Home Management A...,111,2161.0,NaN,1933.0
2,111 Landwirtschaft,"['Agricultural Equipment Operators', 'Farmwork...",9.99,41.500000,111,1,111 Landwirtschaft['Agricultural Equipment Ope...,111,2161.0,NaN,1933.0
3,111 Landwirtschaft,['Agricultural Inspectors'],9.99,68.000000,111,1,111 Landwirtschaft['Agricultural Inspectors'],111,2161.0,NaN,1933.0
4,111 Landwirtschaft,"['Farm and Ranch Managers', 'Aquacultural Mana...",9.99,55.666667,111,1,"111 Landwirtschaft['Farm and Ranch Managers', ...",111,2161.0,NaN,1933.0
...,...,...,...,...,...,...,...,...,...,...,...
331,943 Moderation und Unterhaltung,"['Gaming and Sports Book Writers and Runners',...",19.02,86.500000,943,9,943 Moderation und Unterhaltung['Gaming and Sp...,943,2828.0,NaN,NaN
332,943 Moderation und Unterhaltung,"['Public Address System and Other Announcers',...",19.02,62.500000,943,9,943 Moderation und Unterhaltung['Public Addres...,943,2828.0,NaN,NaN
333,"944 Theater-, Film- und Fernsehproduktion","['Agents and Business Managers of Artists, Per...",18.95,43.000000,944,9,"944 Theater-, Film- und Fernsehproduktion['Age...",944,3792.0,3595.0,NaN
334,"945 Veranstaltungs-, Kamera-, Tontechnik","['Camera Operators, Television, Video, and Mot...",17.61,58.500000,945,9,"945 Veranstaltungs-, Kamera-, Tontechnik['Came...",945,3495.0,3381.0,3233.0


In [29]:
for i in ["Berlin","Brandenburg","Deutschland"]:
    charts=alt.Chart().mark_point().encode(
        x='Nähe',
        y=i,
         tooltip="Bezeichnung_Jobs",
        color="Bezeichnung"
    )
    bakedchart=charts + charts.transform_regression('Nähe',i).mark_line()

    dataset_to_be_used=test
    sectorlist=dataset_to_be_used["Sektor"].unique().tolist()
    sectorlist.sort()
    for j in ["6","8"]:
        print(i)
        subset=dataset_to_be_used[dataset_to_be_used["Sektor"]==j]
        subset=subset[subset["number_to_merge"]!="814"]
        subset=subset.dropna(subset=[i,"Nähe"])
        result=(scipy.stats.linregress(subset[i], subset["Nähe"]))
        print(result.pvalue)
        print(result.rvalue**2)

        combined = alt.layer(charts + charts.transform_regression('Nähe',i).mark_line(), data=subset)
        combined.display()

Berlin
8.199206481364976e-05
0.5133869290771916


alt.LayerChart(...)

Berlin
9.5483884759685e-07
0.36156901896419763


alt.LayerChart(...)

Brandenburg
3.4560178995107406e-05
0.5841854819203016


...

alt.LayerChart(...)

Deutschland
2.7414890286567706e-05
0.5417902336673173


alt.LayerChart(...)

Deutschland
5.132219586408142e-08
0.39249832240740334


alt.LayerChart(...)

I also tried to visualize this only for sector 6 and 8 and highlight the jobs that we had people we talked with. However, we decided to use more simple slopes for our readership in the end.

In [30]:
for i in ["Berlin","Brandenburg","Deutschland"]:
    
    color_scale = alt.Scale(
    domain=[
        "621",
        
    ],
    range=["#c30d24"]
)

    
    charts=alt.Chart().mark_point().encode(
        x='Nähe',
        y=i,
         tooltip="Bezeichnung_Jobs",
           color=alt.condition('datum.number_to_merge==621', alt.ColorValue('orange'), alt.ColorValue('grey'))

    )
    bakedchart=charts + charts.transform_regression('Nähe',i).mark_line()
    
    
    dataset_to_be_used=test
    sectorlist=dataset_to_be_used["Sektor"].unique().tolist()
    sectorlist.sort()
    j="6"
    print(i)
    subset=dataset_to_be_used[dataset_to_be_used["Sektor"]==j]
    subset=subset[subset["number_to_merge"]!="814"]
    subset=subset.dropna(subset=[i,"Nähe"])
    result=(scipy.stats.linregress(subset[i], subset["Nähe"]))
    print(result.pvalue)
    print(result.rvalue**2)

    combined = alt.layer(charts + charts.transform_regression('Nähe',i).mark_line(), data=subset)
    combined.display()

Berlin
8.199206481364976e-05
0.5133869290771916


alt.LayerChart(...)

Brandenburg
3.4560178995107406e-05
0.5841854819203016


alt.LayerChart(...)

Deutschland
2.7414890286567706e-05
0.5417902336673173


alt.LayerChart(...)

In [31]:
for i in ["Berlin","Brandenburg","Deutschland"]:
    
    color_scale = alt.Scale(
    domain=[
        "821",
        
    ],
    range=["#c30d24"]
)

    
    charts=alt.Chart().mark_point().encode(
        x='Nähe',
        y=i,
         tooltip="Bezeichnung_Jobs",
           color=alt.condition('datum.number_to_merge==821', alt.ColorValue('orange'), alt.ColorValue('grey'))

    )
    bakedchart=charts + charts.transform_regression('Nähe',i).mark_line()
    
    
    dataset_to_be_used=test
    sectorlist=dataset_to_be_used["Sektor"].unique().tolist()
    sectorlist.sort()
    j="8"
    print(i)
    subset=dataset_to_be_used[dataset_to_be_used["Sektor"]==j]
    subset=subset[subset["number_to_merge"]!="814"]
    subset=subset.dropna(subset=[i,"Nähe"])
    result=(scipy.stats.linregress(subset[i], subset["Nähe"]))
    print(result.pvalue)
    print(result.rvalue**2)

    combined = alt.layer(charts + charts.transform_regression('Nähe',i).mark_line(), data=subset)
    combined.display()

Berlin
9.5483884759685e-07
0.36156901896419763


alt.LayerChart(...)

Brandenburg
7.2784700868253836e-09
0.4911089303643172


alt.LayerChart(...)

Deutschland
5.132219586408142e-08
0.39249832240740334


alt.LayerChart(...)

In [ ]:
#and then we wrote all of the results to a dataframe
test.to_csv("test.csv",index=False)

# Preparation of slopes
In order to visualize the slopes, I calculated the medians of the tail and head of the dataframes for each jobgroup, sorted by closeness to others

In [50]:
sector6=test[test["Sektor"]=="6"].sort_values(by="Nähe")

In [51]:
sector6.tail(10).median()

Lohn             11.86
Nähe             76.00
Deutschland    2256.50
Berlin         2187.00
Brandenburg    1725.00
dtype: float64

In [52]:
sector6.head(10).median()

Lohn             21.455000
Nähe             48.833333
Deutschland    4185.500000
Berlin         3738.000000
Brandenburg    3418.000000
dtype: float64

In [53]:
sector8_without_doctors=test[(test["number_to_merge"]!="814")&(test["number_to_merge"]!="815")&(test["Sektor"]=="8")].sort_values(by="Nähe")

In [54]:
sector8_without_doctors.tail(10).median()

Lohn             13.29
Nähe             94.00
Deutschland    2434.00
Berlin         2466.50
Brandenburg    2202.00
dtype: float64

In [55]:
sector8_without_doctors.head(10).median()

Lohn             25.24
Nähe             48.00
Deutschland    4551.00
Berlin         4786.00
Brandenburg    3905.00
dtype: float64

In [42]:
sector8_without_doctors[["Bezeichnung_Jobs","Nähe","Deutschland","Berlin","Brandenburg"]].tail(10).to_csv("top10_sector_8.csv")

In [43]:
sector8_without_doctors[["Bezeichnung_Jobs","Nähe","Deutschland"]].head(10).to_csv("low10_sector_8.csv")

In [44]:
sector6[["Bezeichnung_Jobs","Nähe","Deutschland","Berlin","Brandenburg"]].tail(10).to_csv("top10_sector6.csv")

In [45]:
sector6[["Bezeichnung_Jobs","Nähe","Deutschland","Berlin","Brandenburg"]].head(10).to_csv("low10_sector6.csv")

,Bezeichnung_Jobs,Nähe,Deutschland,Berlin,Brandenburg
186,621 Verkauf (ohne Produktspezialisierung)['Ret...,69.0,2630.0,2531.0,2246.0
197,"633 Gastronomie['Food Servers, Nonrestaurant']",72.0,1883.0,1843.0,1725.0
201,"634 Veranstaltungsservice, -management['Meetin...",72.0,2980.0,2985.0,NaN
185,621 Verkauf (ohne Produktspezialisierung)['Gam...,72.5,2630.0,2531.0,2246.0
196,"633 Gastronomie['Counter Attendants, Cafeteria...",74.0,1883.0,1843.0,1725.0
195,633 Gastronomie['Waiters and Waitresses'],78.0,1883.0,1843.0,1725.0
200,"634 Veranstaltungsservice, -management['Hosts ...",79.0,2980.0,2985.0,NaN
198,633 Gastronomie['Bartenders'],79.0,1883.0,1843.0,1725.0
189,631 Tourismus und Sport['Reservation and Trans...,81.0,2634.0,2569.0,2000.0
194,633 Gastronomie['Dining Room and Cafeteria Att...,82.0,1883.0,1843.0,1725.0


# Short-time work
Last step: we want to check whether many of the jobs in sector "6", trade, tourism and food, are currently in short-time work, so that they suffer financially. For this, we need to know whether you can say for the jobs that we are interested in that they mainly work in one branch of economy

In [64]:
#so we call up the data which says how many people from certain job groups are in certain sectors of economy
jobs_to_economy=pd.read_excel("../RawData/berufe_wirtschaftszweige.xlsx", sheet_name="Auswertung_BE", skiprows=6)
jobs_to_economy.columns = jobs_to_economy.iloc[0]
#we cut some unnecessary wors
jobs_to_economy = jobs_to_economy.iloc[5:]
#extract the three digit job code
jobs_to_economy.insert(0, "job_3digits", jobs_to_economy.iloc[:,0].str[:3])
#and additionally rename one of the columns
columns=jobs_to_economy.columns.tolist()
columns[1]="JobGroup"
jobs_to_economy.columns=columns
#then we cut of two unneccessary columns
jobs_to_economy_reshaped=pd.concat([jobs_to_economy.iloc[:,:2],jobs_to_economy.iloc[:,3:-1]],axis=1)
jobs_to_economy_reshaped=jobs_to_economy_reshaped.dropna(subset=["job_3digits"])
#and filter our non-numeric job groups
jobs_to_economy_reshaped=jobs_to_economy_reshaped[jobs_to_economy_reshaped["job_3digits"].str.isnumeric()]
jobs_to_economy_reshaped

,job_3digits,JobGroup,NaN,01,02,03,05,06,07,08,...,90,91,92,93,94,95,96,97,98,99
5,111,111 Landwirtschaft,824.0,23.665,,,,,,,...,,,,,31.1893,,,,,
6,112,112 Tierwirtschaft,38.0,,,,,,,,...,,,,,,,,,,
7,113,113 Pferdewirtschaft,95.0,,,,,,,,...,,,,26.3158,,,,,,
8,114,114 Fischwirtschaft,16.0,,,,,,,,...,,,,,,,,,,
9,115,115 Tierpflege,1039.0,,,,,,,,...,,25.6015,,,11.5496,,2.30991,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,943,943 Moderation und Unterhaltung,532.0,,,,,,,,...,,,46.0526,10.3383,,,,,,
141,944,"944 Theater-, Film- und Fernsehproduktion",2609.0,,,,,,,,...,13.5684,,,0.919893,2.22307,,,,,
142,945,"945 Veranstaltungs-, Kamera-, Tontechnik",4801.0,,,,,,,,...,25.7655,0.333264,,1.04145,1.77046,,0.333264,,,
143,946,"946 Bühnen- und Kostümbildnerei, Requisite",866.0,,,,,,,,...,39.03,,,,,,,,,


In [65]:
#now we turn all columns but the first two into numbers and fillnas with zeroes
jobs_to_economy_reshaped=jobs_to_economy_reshaped.rename(columns={jobs_to_economy_reshaped.columns[2]:"Insgesamt"})

jobs_to_economy_reshaped.iloc[:,2:] = jobs_to_economy_reshaped.iloc[:,2:].apply(pd.to_numeric, errors='coerce')
jobs_to_economy_reshaped=jobs_to_economy_reshaped.fillna(0)
#also we rename one of the columns as it being the number of people in this job
jobs_to_economy_reshaped

,job_3digits,JobGroup,Insgesamt,01,02,03,05,06,07,08,...,90,91,92,93,94,95,96,97,98,99
5,111,111 Landwirtschaft,824.0,23.665049,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,31.189320,0.0,0.000000,0.0,0.0,0.0
6,112,112 Tierwirtschaft,38.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
7,113,113 Pferdewirtschaft,95.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,26.315789,0.000000,0.0,0.000000,0.0,0.0,0.0
8,114,114 Fischwirtschaft,16.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
9,115,115 Tierpflege,1039.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,25.601540,0.000000,0.000000,11.549567,0.0,2.309913,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,943,943 Moderation und Unterhaltung,532.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,46.052632,10.338346,0.000000,0.0,0.000000,0.0,0.0,0.0
141,944,"944 Theater-, Film- und Fernsehproduktion",2609.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,13.568417,0.000000,0.000000,0.919893,2.223074,0.0,0.000000,0.0,0.0,0.0
142,945,"945 Veranstaltungs-, Kamera-, Tontechnik",4801.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,25.765466,0.333264,0.000000,1.041450,1.770464,0.0,0.333264,0.0,0.0,0.0
143,946,"946 Bühnen- und Kostümbildnerei, Requisite",866.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,39.030023,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0


In [66]:
#and we do a sanity check and check whether all percentages add up to hundred
jobs_to_economy_reshaped["sum"]=jobs_to_economy_reshaped[jobs_to_economy_reshaped.columns[3:]].sum(axis=1)
jobs_to_economy_reshaped_with_sum=jobs_to_economy_reshaped
jobs_to_economy_reshaped_with_sum.sort_values(by="sum").head(10)


,job_3digits,JobGroup,Insgesamt,01,02,03,05,06,07,08,...,91,92,93,94,95,96,97,98,99,sum
6,112,112 Tierwirtschaft,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
8,114,114 Fischwirtschaft,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
10,116,116 Weinbau,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
14,211,"211 Berg-, Tagebau und Sprengtechnik",186.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,30.107527
7,113,113 Pferdewirtschaft,95.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,26.315789,0.000000,0.0,0.0,0.0,0.0,0.0,52.631579
135,934,"934 Kunsthandwerkl. Keramik-, Glasgestaltung",83.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,74.698795
124,911,"911 Sprach-, Literaturwissenschaften",218.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,48.623853,0.0,0.0,0.0,0.0,0.0,76.605505
41,291,291 Getränkeherstellung,241.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,80.497925
17,214,"214 Industrielle Keramikherstell.,-verarbeit",101.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,81.188119
134,933,933 Kunsthandwerk und bildende Kunst,391.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,17.647059,0.0,0.000000,2.557545,0.0,0.0,0.0,0.0,0.0,81.329923


We do not get a hundred because some job groups are just too small to be still shown in the statistic because it would make either employers or employees identifiable. Now we want to check for sector 6, how this is divided between different sectors of economy

In [48]:
#we filter for only job sector 6
only_tourism_gastro=jobs_to_economy_reshaped_with_sum[jobs_to_economy_reshaped_with_sum["job_3digits"].str[0]=="6"]
only_tourism_gastro

,job_3digits,JobGroup,Insgesamt,01,02,03,05,06,07,08,...,91,92,93,94,95,96,97,98,99,sum
80,611,611 Einkauf und Vertrieb,33074.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.069541,0.217694,0.341658,0.671222,0.102800,0.568422,0.000000,0.0,0.000000,99.612989
81,612,612 Handel,3113.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.353357,95.277867
82,613,"613 Immobilienwirtschaft,Facility-Management",8387.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.119232,0.000000,0.119232,0.798855,0.000000,0.131155,0.000000,0.0,0.000000,98.771909
83,621,621 Verkauf (ohne Produktspezialisierung),59836.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.152082,0.564877,0.452905,0.426165,0.091918,0.427836,0.000000,0.0,0.000000,99.893041
84,622,"622 Verkauf Bekleid.,Elektro,KFZ,Hartwaren",17121.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.169383,0.075930,0.000000,0.0,0.000000,99.474330
85,623,623 Verkauf von Lebensmitteln,10936.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.173738,0.000000,0.000000,0.274323,0.000000,0.0,0.000000,99.478786
86,624,"624 Verkauf drog.apotheken.Waren,Medizinbed.",5036.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.814138,0.000000,0.0,0.000000,99.503574
87,625,"625 Buch-Kunst-Antiquitäten-,Musikfachhandel",863.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.939745,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,96.871379
88,631,631 Tourismus und Sport,4280.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.897196,0.000000,16.448598,2.242991,0.000000,0.514019,0.000000,0.0,0.000000,97.429907
89,632,632 Hotellerie,15227.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.151047,1.031063,0.492546,0.000000,2.883037,0.085375,0.0,0.098509,99.494319


Here we check which is the sector of economy in which most people of a certain job group are employed in

In [67]:
max_sector=only_tourism_gastro[only_tourism_gastro.columns[3:-1]].idxmax(axis=1)
max_value=only_tourism_gastro[only_tourism_gastro.columns[3:-1]].max(axis=1)
frontcolumns=only_tourism_gastro[only_tourism_gastro.columns[:3]]
jobgroup_to_max_sector=pd.concat([frontcolumns,max_value,max_sector],axis=1)
jobgroup_to_max_sector

,job_3digits,JobGroup,Insgesamt,0,1
80,611,611 Einkauf und Vertrieb,33074.0,14.745722,46
81,612,612 Handel,3113.0,56.537102,46
82,613,"613 Immobilienwirtschaft,Facility-Management",8387.0,76.082032,68
83,621,621 Verkauf (ohne Produktspezialisierung),59836.0,78.384250,47
84,622,"622 Verkauf Bekleid.,Elektro,KFZ,Hartwaren",17121.0,76.298113,47
85,623,623 Verkauf von Lebensmitteln,10936.0,57.818215,47
86,624,"624 Verkauf drog.apotheken.Waren,Medizinbed.",5036.0,91.938046,47
87,625,"625 Buch-Kunst-Antiquitäten-,Musikfachhandel",863.0,83.545771,47
88,631,631 Tourismus und Sport,4280.0,54.789720,79
89,632,632 Hotellerie,15227.0,56.990871,55


So now we know for instance, that 68 percent of people in gastronomy work in economy sector 56. Now we want to know how many people are currently on short-work scheme. First, we load in a file with how many people are employed in the economy sectors.

In [68]:
#now we want to know f
wirtschaftszweige_be=pd.read_excel("../RawData/beschaeftigte_nach_wirtschaftszweigen/wz-heft-11-0-201909-xlsx.xlsx", sheet_name="SVB - Tabelle I", skiprows=7)
wirtschaftszweige_be["Sektor"]=wirtschaftszweige_be["Wirtschaftsabteilungen und -gruppen der WZ 2008"].str[:2]
wirtschaftszweige_be["uwz"]=wirtschaftszweige_be["Wirtschaftsabteilungen und -gruppen der WZ 2008"].str[2]

wirtschaftszweige_be=wirtschaftszweige_be.dropna(subset=["Sektor"])
wirtschaftszweige_be=wirtschaftszweige_be[wirtschaftszweige_be["Sektor"].str.isdigit()]
wirtschaftszweige_be=wirtschaftszweige_be[~wirtschaftszweige_be["uwz"].str.isdigit()]

wirtschaftszweige_be

,Wirtschaftsabteilungen und -gruppen der WZ 2008,Insgesamt,darunter,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,darunter: Auszubildende,Unnamed: 12,Unnamed: 13,Sektor,uwz
5,"01 Landwirtschaft, Jagd und damit verbundene T...",633,384,249,479,153,41,495,88,9,3,20,12,8,01,
13,02 Forstwirtschaft und Holzeinschlag,9,*,*,9,0,*,6,*,*,0,0,0,0,02,
18,03 Fischerei und Aquakultur,*,*,*,*,*,*,*,*,*,*,*,*,*,03,
21,05 Kohlenbergbau,0,0,0,0,0,0,0,0,0,0,0,0,0,05,
24,06 Gewinnung von Erdöl und Erdgas,*,*,*,*,*,*,*,*,*,*,*,*,*,06,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,95 Rep.v. DV-geräten und Gebrauchsgütern,1572,1140,432,1306,258,109,1096,346,21,7,53,42,11,95,
356,96 Erbr.v.sonstigen überwiegend persönlichen D...,16195,4714,11481,11970,4124,1679,11545,2680,291,59,849,304,545,96,
358,97 Private Haushalte mit Hauspersonal,2259,471,1788,1607,639,74,1435,660,90,24,0,0,0,97,
360,"98 H.v.Waren,Dienstl.d.priv. Haushalte oaS",44,19,25,34,8,*,32,*,0,0,0,0,0,98,


Then we load a file that lists up how many people are currently in short-work scheme in each sector

In [69]:

kurzarbeit_be=pd.read_excel("../RawData/kurzarbeit.xlsx", sheet_name="Berlin", skiprows=0)
kurzarbeit_be["Kat"]=kurzarbeit_be["Kat"].str.replace(' ', '')
kurzarbeit_be["Sektor"]=kurzarbeit_be["Kat"].str[:2]
kurzarbeit_be=kurzarbeit_be.dropna(subset=["Sektor"])
kurzarbeit_be=kurzarbeit_be[kurzarbeit_be["Sektor"].str.isnumeric()]
kurzarbeit_be

,SUM,Kat,Sektor
1,114.0,"01Landwirtsch.,Jagdu.damitverb.Tätigk.",01
2,0.0,02ForstwirtschaftundHolzeinschlag,02
3,0.0,03FischereiundAquakultur,03
4,0.0,05Kohlenbergbau,05
5,0.0,06GewinnungvonErdölundErdgas,06
...,...,...,...
84,649.0,95Reparaturv.DV-Gerätenu.Gebrauchsgütern,95
85,9860.0,96Sonstigeüberwieg.persönlicheDL,96
86,46.0,97PrivateHaushaltemitHauspersonal,97
87,0.0,98DL+Herst.v.Warend.PHf.Eigenbed.,98


And then we calculate which share is working in short work scheme

In [71]:
#we merge and calculate the ratio of people in kurzarbeit
be_merge=pd.merge(kurzarbeit_be,wirtschaftszweige_be, how="inner",on="Sektor")
be_merge["ratio_short_work"]=be_merge["SUM"]/pd.to_numeric(be_merge["Insgesamt"],errors="coerce")
be_merge=be_merge.dropna()
be_merge.sort_values(by="ratio_short_work").tail(50)
#be_merge[be_merge["Sektor"]=="47"]

,SUM,Kat,Sektor,Wirtschaftsabteilungen und -gruppen der WZ 2008,Insgesamt,darunter,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,darunter: Auszubildende,Unnamed: 12,Unnamed: 13,uwz,ratio_short_work
50,2548.0,58Verlagswesen,58,58 Verlagswesen,12519,6553,5966,10690,1824,687,10207,1521,104,46,305,134,171,,0.203531
62,6824.0,"71Architektur-,Ingenieurbüros;Labore",71,"71 Architektur- und Ingenieurbüros; techn., ph...",33445,19894,13551,28595,4805,1527,25956,5379,583,155,433,299,134,,0.204036
21,530.0,"23Hrst.v.Glas,Keramik,Verarb.Steine+Erden",23,"23 Herst.v. Glas u.Glaswaren, Keramik, Verarb....",2597,1937,660,2417,179,191,1779,596,31,10,111,80,31,,0.204082
55,3971.0,63Informationsdienstleistungen,63,63 Informationsdienstleistungen,18799,10702,8097,13651,5131,1134,16599,1021,45,21,113,71,42,,0.211235
75,23835.0,86Gesundheitswesen,86,86 Gesundheitswesen,112627,25685,86942,103128,9392,9916,81046,20073,1592,424,5963,838,5125,,0.211628
26,1724.0,28Maschinenbau,28,28 Maschinenbau,7218,5895,1323,6690,524,414,4832,1875,97,36,289,257,32,,0.238847
29,346.0,31HerstellungvonMöbeln,31,31 Herstellung von Möbeln,1326,1056,270,1202,123,214,920,180,12,0,207,169,38,,0.260935
22,341.0,24Metallerzeugungund-bearbeitung,24,24 Metallerzeugung und -bearbeitung,1299,1114,185,1110,186,86,840,358,15,4,37,*,*,,0.262510
79,1379.0,"91Bib.,Archive,Museen,botan.u.zoologGärt.",91,"91 Bibl.,Archive,Museen,zoolog.u.ä.Gärten",5248,2092,3156,4917,330,266,3352,1513,117,38,129,43,86,,0.262767
31,901.0,33Rep.u.Install.v.Masch.u.Ausrüstungen,33,33 Reparatur und Installation von Maschinen un...,3204,2654,550,2851,351,183,2205,756,60,18,117,102,15,,0.281211


In [72]:
#we reduce this to the essential columns
benennung=be_merge[["Sektor","Wirtschaftsabteilungen und -gruppen der WZ 2008","ratio_short_work"]]
benennung

,Sektor,Wirtschaftsabteilungen und -gruppen der WZ 2008,ratio_short_work
0,01,"01 Landwirtschaft, Jagd und damit verbundene T...",0.180095
1,02,02 Forstwirtschaft und Holzeinschlag,0.000000
8,10,10 Herstellung von Nahrungs- und Futtermitteln,0.365383
9,11,11 Getränkeherstellung,0.458373
11,13,13 Herstellung von Textilien,0.689003
...,...,...,...
83,95,95 Rep.v. DV-geräten und Gebrauchsgütern,0.412850
84,96,96 Erbr.v.sonstigen überwiegend persönlichen D...,0.608830
85,97,97 Private Haushalte mit Hauspersonal,0.020363
86,98,"98 H.v.Waren,Dienstl.d.priv. Haushalte oaS",0.000000


In [73]:
#and then we merge this onto the previous dataframe for sector 6. 
pd.merge(benennung,jobgroup_to_max_sector,left_on="Sektor",right_on=1)

,Sektor,Wirtschaftsabteilungen und -gruppen der WZ 2008,ratio_short_work,job_3digits,JobGroup,Insgesamt,0,1
0,46,46 Großhandel (ohne Handel mit Kraftfahrzeugen),0.347908,611,611 Einkauf und Vertrieb,33074.0,14.745722,46
1,46,46 Großhandel (ohne Handel mit Kraftfahrzeugen),0.347908,612,612 Handel,3113.0,56.537102,46
2,47,47 Einzelhandel (ohne Handel mit Kraftfahrzeugen),0.283843,621,621 Verkauf (ohne Produktspezialisierung),59836.0,78.384250,47
3,47,47 Einzelhandel (ohne Handel mit Kraftfahrzeugen),0.283843,622,"622 Verkauf Bekleid.,Elektro,KFZ,Hartwaren",17121.0,76.298113,47
4,47,47 Einzelhandel (ohne Handel mit Kraftfahrzeugen),0.283843,623,623 Verkauf von Lebensmitteln,10936.0,57.818215,47
5,47,47 Einzelhandel (ohne Handel mit Kraftfahrzeugen),0.283843,624,"624 Verkauf drog.apotheken.Waren,Medizinbed.",5036.0,91.938046,47
6,47,47 Einzelhandel (ohne Handel mit Kraftfahrzeugen),0.283843,625,"625 Buch-Kunst-Antiquitäten-,Musikfachhandel",863.0,83.545771,47
7,55,55 Beherbergung,0.832412,632,632 Hotellerie,15227.0,56.990871,55
8,56,56 Gastronomie,0.790742,633,633 Gastronomie,43192.0,68.605297,56
9,68,68 Grundstücks- und Wohnungswesen,0.137453,613,"613 Immobilienwirtschaft,Facility-Management",8387.0,76.082032,68


this shows that 68 percent of people with a job in gastronomy also work in the economic sector gastronomy (some people might work in there as a trade expert, account, this is why). and in this economic sector, currently 79 percent of the staff are in short-work scheme. hence, the 79 percent.